# <span style="color:blue">Importing libraries</span>

In [ ]:
# import pandas as pd
# import numpy as np
# import scipy.stats as st

# import seaborn as sns
# import plotly.express as px
# import matplotlib.pyplot as plt
# import ipyplot

# import requests
# import os.path
# import re

# <span style="color:blue">Data reading from local files</span>

In [ ]:
ufo = pd.read_csv('ufo_for_analysis.csv', index_col=0, parse_dates=['Observed', 'Posted'])

states = pd.read_csv('states.csv', index_col=0)

# <span style="color:blue">Exploratory data analysis</span>

As the first step, let us first review **the distribution of the observed UFO by shape**.

From the plot below it is to be seen that the most frequently observed objects are of **round** shape or just **lights**. Other shapes are less common, and some specific shapes like hexagon or pyramid were observed just several times.

It is important to notice that quite many of the observations (17.8%) did not state the shape of the object.

On top of incomplete reports, this may come from the fact that about 1.3% of the observations contain ["MADAR"](https://madar.site/#home) or "RADAR" in the `Summary` field and hence come from some automated observation systems which track the appearance of something yet cannot determine the shape of the object.

In [ ]:
ufo_per_shape = ufo.groupby('Shape')\
        .agg(percentage =('Summary', lambda x: x.size / ufo.shape[0] * 100))\
        .sort_values(by='percentage', ascending=False)\
        .round(2)

idx = ufo_per_shape.index.tolist()
idx.remove('unknown')
ufo_per_shape = ufo_per_shape.reindex(idx + ['unknown'])

sns.set_style("whitegrid")
sns.set(font_scale=1.1)

bar,ax = plt.subplots(figsize=(12,8))
ax = sns.barplot(x=ufo_per_shape.index,
                 y='percentage',
                 data=ufo_per_shape,
                 ci=None,
                 palette="muted",
                 orient='v', )
ax.set_title("Distribution of UFO observations by the shape", fontsize=15)
ax.set_xlabel ("UFO shape")
ax.set_ylabel ("Percentage")
ax.tick_params(axis='x', rotation=45)

# calculate the percentages and annotate the sns barplot
for rect in ax.patches:
    ax.text(rect.get_x(),rect.get_height() + 0.1,"%.1f%%"% rect.get_height(), weight='bold' );

In [ ]:
round(ufo.query('Summary\
                .str.upper()\
                .str.contains("[M|R]ADAR", na=False)')\
          .shape[0] 
    / ufo.shape[0], 3)

Now let us consider a **general trend in the number of observed UFO**.

Note that hereafter we only consider the cases with a filled `Summary` field, meaning that a case without anything described cannot be verified even theoretically and is not therefore a valid case. Another possibility is to count the entries in the `Posted` field, yet this does not affect the general conclusions.

The plot below reveals several characteristic ranges:
* Before ~1960, the cases of UFO observation are fairly rare. This is not strange: it is stated in the [web page]() that the National UFO Reporting Center is in operation since 1974, and earlier reports have been likely filed basing on some scarce reports.
* Over 1960-1990, the number of UFO observations has remained relatively low. During this period, the cases were likely collected by post or like this.
* The onset of steep growth in the number of UFO observations is around 1995, which corresponds to the appearance of the online database (the earliers `Posted` date is March, 1998, see the query below).
* A remarkable feature is general decrease in the total observations number after 2016 and sudden increase in 2020 (the latter effect may be somehow related to the COVID-19 lockdown).

It is important to notice that the historical data in this database may somewhat change in furute. For example, the case observed at 1905-07-04 15:00:00 was filed just a month before this analysis, on 2021-07-31 (see below). However, this will not likely affect the general conclusions.

In [ ]:
ufo_per_year = ufo.groupby(ufo.Observed.dt.year)\
                .Summary\
                .count()

sns.set_style("whitegrid")
sns.set(font_scale=1.1)

plt.figure(figsize=(12,8))
ax = sns.lineplot(x=ufo_per_year.index.astype('int'),
                 y=ufo_per_year,
                 palette="muted")

ax.set_title("Total UFO observed by year", fontsize=15)
ax.set_xlabel ("Year")
ax.set_ylabel ("Observations");

In [ ]:
ufo_early_posted = ufo.sort_values(by='Posted')\
                      .head()

ufo_early_posted\
    .style\
    .set_properties(**{'background-color': '#ffffb3'}, subset=['Posted'])

In [ ]:
def highlight_early(s, props=''):
    return np.where(s < '1906-01-01', props, '')

def highlight_late(s, props=''):
    return np.where(s > '2021-01-01', props, '')

ufo_early_observations = ufo.query('~Observed.isna()')\
                .sort_values(by='Observed')\
                .head()

ufo_early_observations\
        .style\
        .apply(highlight_early, props='color:red;', axis=0, subset=['Observed'])\
        .apply(highlight_late, props='color:blue;', axis=0, subset=['Posted'])

Now let us consider the historical **trends in the observations of different shapes of UFO**.

For better view, we have started the plot with year of 1960 and the UFO shapes to several most popular ones. Note the logarithmic scale of the number of observations.

It is to be seen that in general the trends are similar, with the expection of "Fireball" shape which shows a pair of sudden jumps. However, in view of 'multiple comparison' issue, it is fair to assume that there have been no anomalous changes in the appearance of UFO of certain shape over the reported cases.

In [ ]:
ufo_per_year_shape = ufo.assign(Year = ufo.Observed.dt.year)

ufo_per_year_shape = ufo_per_year_shape.groupby(['Year', 'Shape'], as_index=False)\
        .agg({'Summary': 'count'})\
        .pivot('Year', columns='Shape', values='Summary')

ufo_per_year_shape['overall'] = ufo_per_year_shape.sum(axis=1)

ufo_per_year_shape = ufo_per_year_shape.reset_index()\
    .melt(id_vars = 'Year', var_name='Shape',  value_name='Observations')

ufo_per_year_shape = ufo_per_year_shape.query("Shape in ['overall', 'circle', 'light', 'triangle', 'fireball', 'oval', 'cylinder', 'formation']")\
            .query("1960 <= Year <= 2021")


sns.set_style("whitegrid")
sns.set(font_scale=1.1)

plt.figure(figsize=(12,8))
ax = sns.lineplot(data=ufo_per_year_shape,
                  x='Year',
                  y='Observations',
                  hue='Shape',
                 palette="muted")

ax.set_title("UFO observed by year", fontsize=15)
ax.set_xlabel ("Year")
ax.set_ylabel ("Observations")
ax.set_yscale("log");

Let us consider the distribution of the cases of **UFO observations over the days of the week**.

Seemingly, the number of such follows certain cycle with a peak on Saturdays. However, the difference is not huge, and moreover the total counts accumulate the cases from quite long period. Therefore, it seems reasonable to run a statistical test to verify the significance of this observation.

In [ ]:
ufo_per_day = ufo.groupby(ufo.Observed.dt.day_name())\
                .Summary\
                .count()\
                .reindex(index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

sns.set_style("whitegrid")
sns.set(font_scale=1.1)

plt.figure(figsize=(12,8))
ax = sns.barplot(x=ufo_per_day.index,
                 y=ufo_per_day,
                 palette="muted")

ax.set_title("Total UFO observed by day", fontsize=15)
ax.set_xlabel ("Day")
ax.set_ylabel ("Observations");

A straightforward day to do so is to consider the difference between the number of UFO observations on Saturday and the rolling mean over 3 days (which a week centered on Saturday).

Note that this is one of the simplest methods to do so, yet it gives quite convincing results.

In [ ]:
ufo_daily_count = ufo.resample(rule='D', on='Observed').Summary.count()
ufo_rolling_mean = ufo.resample(rule='D', on='Observed').Summary.count().rolling(window=3, center=True).mean()

ufo_daily_diff = ufo_daily_count - ufo_rolling_mean

diff_Saturday = ufo_daily_diff[ufo_daily_diff.index.day_name() == 'Saturday']

diff_Wednesday = ufo_daily_diff[ufo_daily_diff.index.day_name() == 'Wednesday']

The obtained difference values can be tested against a null hypothesis of equality to zero.

The corresponding $p$-value of the single-sample $t$-test is of 5e-18, which leads to rejection of the null hypothesis and confirms that the difference in the UFO observation cases on Saturdays is significantly higher than during the other days. As a comparison, the same statistics calculated for Wednesdays gives $p$-value of 0.9, hence we fail to reject the null hypothesis at any reasonable significance level.

In [ ]:
diff_Saturday.mean().round(4)

In [ ]:
st.ttest_1samp(diff_Saturday, popmean=0, alternative='two-sided')

In [ ]:
diff_Wednesday.mean().round(4)

In [ ]:
st.ttest_1samp(diff_Wednesday, popmean=0, alternative='two-sided', nan_policy='omit')

The distribution of the **observed cases with respect to the time** seems reasonable: most often, UFO are observed 6 pm and 1 am, when it is dark yet people are still active.

In [ ]:
ufo_per_hour = ufo.groupby(ufo.Observed.map(lambda t: t.hour))\
                .Summary\
                .count()

sns.set_style("whitegrid")
sns.set(font_scale=1.1)

plt.figure(figsize=(12,8))
ax = sns.barplot(x=ufo_per_hour.index.astype('int'),
                 y=ufo_per_hour,
                 palette="muted")

ax.set_title("Total UFO observed by time", fontsize=15)
ax.set_xlabel ("Hour")
ax.set_ylabel ("Observations");

Let us now look at the **observations cases per state** (including these of US and Canada).

It is to be seen that the distribution of the cases is strongly uneven: it ranges between just 1 (for VI, Virgin Islands) and 14806 for CA (California).

In [ ]:
ufo_per_state = ufo.groupby('State', as_index=False)\
        .agg({'Summary': 'count'})\
        .rename(columns={'Summary': 'Observations'})\
        .sort_values(by='State', ascending=True)

#Seaborn barplot
sns.set_style("whitegrid")

sns.set(font_scale=1)

bar,ax = plt.subplots(figsize=(14,10))
ax = sns.barplot(x='State',
                 y='Observations',
                 data=ufo_per_state,
                 ci=None,
                 palette="muted",
                 orient='v')
ax.set_title("Distribution of UFO observed by state", fontsize=15)
ax.set_xlabel ("State")
ax.set_ylabel ("Number of observations")
ax.tick_params(axis='x', rotation=90)

In [ ]:
ufo_per_state.describe()

In [ ]:
pd.concat([ufo_per_state[ufo_per_state.Observations == min(ufo_per_state.Observations)],
           ufo_per_state[ufo_per_state.Observations == max(ufo_per_state.Observations)]])

The simplest idea behind the observed difference is that the states are strongly different in area and population, both factors being potentially in favor of more frequent observations of whatever phenomenon occurring randomly.

Therefore, let us consider the similar plots upon normalization of the total observations counts with respect to the states population and area.

In [ ]:
ufo_per_state = pd.merge(left=ufo_per_state, 
                         right=states, how='inner', 
                         left_on='State', 
                         right_on='Abbreviation')

In [ ]:
ufo_per_state['Observations_per_population'] = ufo_per_state.Observations / ufo_per_state.Population
ufo_per_state['Observations_per_area'] = ufo_per_state.Observations / ufo_per_state.Total_area
ufo_per_state['Observations_per_land_area'] = ufo_per_state.Observations / ufo_per_state.Land_area

The distribution of the **observation cases normalized by the population** (as per current data from Wikipedia) becomes more uniform in the range of the highest values, still, there are some states with extremely low observation frequency.

In [ ]:
sns.set_style("whitegrid")

sns.set(font_scale=1)

bar,ax = plt.subplots(figsize=(14,10))
ax = sns.barplot(x='State',
                 y='Observations_per_population',
                 data=ufo_per_state,
                 ci=None,
                 palette="muted",
                 orient='v')
ax.set_title("Distribution of UFO observed by state normalized by current population", fontsize=15)
ax.set_xlabel ("State")
ax.set_ylabel ("Number of observations per person")
ax.tick_params(axis='x', rotation=90)

However, the distribution of the **observation cases normalized by land area** (the data normalized by total area are very similar and not shown for clarity) is strikingly different.

There is a state (actually, District of Columbia) with extremely high normalized observations frequency.

This fact definitely comes from very small area of DC - even though the total number of observations in DC is as low as 139, 10th percentile of the data, it returns huge normalized value with respect to very small area.

In [ ]:
sns.set_style("whitegrid")

sns.set(font_scale=1)

bar,ax = plt.subplots(figsize=(14,10))
ax = sns.barplot(x='State',
                 y='Observations_per_land_area',
                 data=ufo_per_state,
                 ci=None,
                 palette="muted",
                 orient='v')
ax.set_title("Distribution of UFO observed by state normalized by land area", fontsize=15)
ax.set_xlabel ("State")
ax.set_ylabel ("Number of observations per sq.km")
ax.tick_params(axis='x', rotation=90)

In [ ]:
ufo_per_state.query('State == "DC"')

In [ ]:
st.percentileofscore(ufo_per_state.Observations, 139)

To better describe the obtained normalized data, let us consider the **distributions of the standartized values**.

In [ ]:
def z_score(column):
    column.replace([np.inf, -np.inf], np.nan, inplace=True)
    return (column - column.mean())/column.std()

ufo_per_state['Observations_std'] = z_score(ufo_per_state['Observations'])
ufo_per_state['Observations_per_population_std'] = z_score(ufo_per_state['Observations_per_population'])
ufo_per_state['Observations_per_area_std'] = z_score(ufo_per_state['Observations_per_area'])
ufo_per_state['Observations_per_land_area_std'] = z_score(ufo_per_state['Observations_per_land_area'])

std_observations = ufo_per_state.loc[:, ['State', 'Observations_std', 'Observations_per_population_std', 'Observations_per_area_std', 'Observations_per_land_area_std']]

std_observations = pd.melt(frame=std_observations, 
        id_vars='State',
        value_vars=['Observations_std', 'Observations_per_population_std', 'Observations_per_area_std', 'Observations_per_land_area_std'],
        )

It is to be seen that the initial distribution of the total observation cases by state is somewhat skewed, as marked by a few outliers in the range of high number of observations.

These outliers have disappeared upon normalization by current population, yet the variance of the data (as estimated from the interquartile range) has been somewhat widened.

The distributions upon normalization to the state area are very narrow, except for the outlier value for DC.

In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(x="value", y="variable", data=std_observations, color="c")
sns.despine(trim=True)

Wider distribution of the values normalized by population with respect to those normalized by area is likely due to the fact that the current population has been taken as reference, even though the data spans more than a century. From the historical data on the USA population it is to be seen that the population change has been different in different states: for some of them, it remained almost constant over recent decades, being growing rapidly for others. For sure, this imparts a deal of additional variation reflected in the wider distribution.

In [ ]:
ipyplot.plot_images(
    ['https://upload.wikimedia.org/wikipedia/commons/f/f3/US_state_historical_population_FRED_SMIL.svg'],
    img_width=800);

Let us return for a moment to **a special case of DC**.

One could imagine that this extremely high frequency of the UFO observations when normalized to the state area is due to the fact thet the District of Columbia is at the very center of some anomaly zone, whereas the potentially frequent observations in the neighbor states could be masked by their relatively large area.

In such case, the states adjacent to DC would show somewhat higher frequency of the observations, yet not high enough to be shown as outlier.

The map of the values of the cases of UFO observations normalized by the state area shown below does not show any specific pattern (zoom in to spot the DC as a yellow-colored region).

In [ ]:
fig = px.choropleth(locations=std_observations.query("variable == 'Observations_per_area_std'")\
                                              .State,
                    locationmode="USA-states",
                    color=std_observations.query("variable == 'Observations_per_area_std'")\
                                           .value,
                    scope="usa")
fig.show()

Removal of the value for DC has allowed visualization of certain pattern.

Note that even though the states with the highest UFO count normalized by the area are grouped in the Eastern part of the USA, the states adjacent to DC do not reveal the highest normalized UFO counts.

In [ ]:
fig = px.choropleth(locations=std_observations.query("variable == 'Observations_per_area_std'")\
                                              .query("abs(value) < 1")\
                                              .State,
                    locationmode="USA-states",
                    color=std_observations.query("variable == 'Observations_per_area_std'")\
                                           .query("abs(value) < 1")\
                                           .value,
                    scope="usa")
fig.show()

Similarly, the map of the UFO counts normalized by population has not shown any specific pattern.

In [ ]:
fig = px.choropleth(locations=std_observations.query("variable == 'Observations_per_population_std'").State,
                    locationmode="USA-states",
                    color=std_observations.query("variable == 'Observations_per_population_std'").value,
                    scope="usa",
                    title='Z-score of UFO observations per current state population')
fig.show()

As a final point, let us compare the USA and Canada states with the DC data alone to see any specific features.

In [ ]:
us_states = states.iloc[:56]
canada_states = states.iloc[56:]

In [ ]:
ufo_us = ufo.query('State in @us_states.Abbreviation')
ufo_canada = ufo.query('State in @canada_states.Abbreviation')
ufo_dc = ufo_us.query('State == "DC"')
ufo_no_dc = ufo_us.query('State != "DC"')

In [ ]:
ufo_us_per_hour = ufo_us.groupby(ufo_us.Observed.map(lambda t: t.hour))\
                .Summary\
                .count()

ufo_canada_per_hour = ufo_canada.groupby(ufo_canada.Observed.map(lambda t: t.hour))\
                .Summary\
                .count()

ufo_dc_per_hour = ufo_dc.groupby(ufo_dc.Observed.map(lambda t: t.hour))\
                .Summary\
                .count()

ufo_no_dc_per_hour = ufo_no_dc.groupby(ufo_no_dc.Observed.map(lambda t: t.hour))\
                .Summary\
                .count()


ufo_us_per_hour = pd.DataFrame({
    'Observations': ufo_us_per_hour / ufo_us_per_hour.sum(),
    'Region': 'US'
            })

ufo_canada_per_hour = pd.DataFrame({
    'Observations': ufo_canada_per_hour / ufo_canada_per_hour.sum(),
    'Region': 'Canada'
            })

ufo_dc_per_hour = pd.DataFrame({
    'Observations': ufo_dc_per_hour / ufo_dc_per_hour.sum(),
    'Region': 'DC'
            })

ufo_no_dc_per_hour = pd.DataFrame({
    'Observations': ufo_no_dc_per_hour / ufo_no_dc_per_hour.sum(),
    'Region': 'US outside DC'
            })



ufo_region_hour = pd.concat([ufo_canada_per_hour, ufo_dc_per_hour, ufo_no_dc_per_hour])


In [ ]:
sns.set_style("whitegrid")
sns.set(font_scale=1.1)

plt.figure(figsize=(12,8))
ax = sns.lineplot(x=ufo_region_hour.index.astype('int'),
                 y=ufo_region_hour.Observations,
                 hue=ufo_region_hour.Region, 
                 palette="muted")

ax.set_title("Total UFO observed by time", fontsize=15)
ax.set_xlabel ("Hour")
ax.set_ylabel ("Observations fraction");

In [ ]:
ufo_us_per_shape = ufo_us.groupby('Shape')\
        .agg(percentage =('Summary', lambda x: x.size / ufo_us.shape[0] * 100))\
        .sort_values(by='percentage', ascending=False)\
        .round(2)
ufo_us_per_shape['Region'] = 'US'


ufo_canada_per_shape = ufo_canada.groupby('Shape')\
        .agg(percentage =('Summary', lambda x: x.size / ufo_canada.shape[0] * 100))\
        .sort_values(by='percentage', ascending=False)\
        .round(2)
ufo_canada_per_shape['Region'] = 'Canada'

ufo_dc_per_shape = ufo_dc.groupby('Shape')\
        .agg(percentage =('Summary', lambda x: x.size / ufo_dc.shape[0] * 100))\
        .sort_values(by='percentage', ascending=False)\
        .round(2)
ufo_dc_per_shape['Region'] = 'DC'

ufo_no_dc_per_shape = ufo_no_dc.groupby('Shape')\
        .agg(percentage =('Summary', lambda x: x.size / ufo_no_dc.shape[0] * 100))\
        .sort_values(by='percentage', ascending=False)\
        .round(2)
ufo_no_dc_per_shape['Region'] = 'US outside DC'

ufo_region_shape = pd.concat([ufo_canada_per_shape, ufo_dc_per_shape, ufo_no_dc_per_shape])

In [ ]:
idx = ufo_per_shape.index.tolist()
idx.remove('unknown')
idx.append('unknown')



sns.set_style("whitegrid")
sns.set(font_scale=1.1)

bar,ax = plt.subplots(figsize=(12,8))
ax = sns.barplot(x=ufo_region_shape.index,
                 y='percentage',
                 data=ufo_region_shape,
                 hue='Region',
                 ci=None,
                 palette="muted",
                 orient='v',
                 order=idx)
ax.set_title("Distribution of UFO observations by the shape", fontsize=15)
ax.set_xlabel ("UFO shape")
ax.set_ylabel ("Percentage")
ax.tick_params(axis='x', rotation=45)

# calculate the percentages and annotate the sns barplot
# for rect in ax.patches:
#     ax.text(rect.get_x(),rect.get_height() + 0.1,"%.1f%%"% rect.get_height(), weight='bold' );

It is to be seen that the distributions of the UFO observations over time of a day is almost identical for District of Columbia, USA, and Canada.

Similarly, no significant differrences have been founf in the distribution of UFO by shape between these regions.

The most striking difference has been revealed in the percentage of cylinder-shaped UFO (about 7% in DC and about 4% in other regions), yet 3% difference of 139 cases observed in DC is just 4 to 5 cases over more than a century of the data collection, and this unlikely marks anything special.

# <span style="color:blue">Overall conclusion and future directions</span>

Even though personally I do not believe in 'alien' origin of UFO, I think that certain natural or technical phenomena can be regarded as such. 

The data obtained from the [http://www.nuforc.org/](http://www.nuforc.org/) web page give many inspirations for analysis, apart from the alien hypothesis.

Even preliminary exploratory consideration of the available data has shown that the observation of UFO is likely a general worldwide phenomenon which can be investigated further in connection with different political, health, and cultural ussues possibly affecting the desire for people to observe something 'strange'.

Moreover, this dataset has much more to discover, including possibly the NLP analysis of the `Summary` field in the data to reveal common features and their historical trends.